In [1]:
from ehrmonize import EHRmonize
import pandas as pd
import os

In [3]:
prompt = "You are a well trained clinician doing data cleaning and harmonization. \
            You are given a raw drug name out of EHR, below, within squared brackets[]. \
            Please give me this drug's generic name in accordance with RxNorm standards: [0.9% Sodium Chloride] \
            Remove salt names unless there are multiple salts with different clinical effects. \
            Remove prescription strengths. \
            Include concentrations for intravenous fluids and dextrose solutions. \
            Please output nothing more than the generic name in lowercase."

In [3]:
models = [
    # 'gpt-4o',
    # 'meta.llama3-70b-instruct-v1:0',
    # 'mistral.mixtral-8x7b-instruct-v0:1',
    # 'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [ ]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=1,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='classify_drug',
            classes=['antibiotic', 'sedative', 'other'],
            metrics=['accuracy_score']
      )

      pred = ehrm.predict(
        pd.DataFrame({
            'drug': ['ativan','vancocin'],
            'route': ['intravenous','oral']
        })
      )['pred']

      acc = ehrm.evaluate(
            pd.Series(['sedative','antibiotic']),
            pred
      )

      print(pred)
      print(acc)



In [4]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=1,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='classify_drug',
            classes=['antibiotic', 'sedative', 'other'],
            metrics=['accuracy_score']
      )

      pred = ehrm.predict(
        pd.DataFrame({
            'drug': ['ativan','vancocin'],
            'route': ['intravenous','oral']
        })
      )

      acc = ehrm.evaluate(
            pd.Series(['sedative','antibiotic']),
            pred
      )

      print(pred)
      print(acc)



0      sedative
1    antibiotic
dtype: object
1.0


In [5]:
possible_shots = [
    "An input route 'intraven' would be classified as 'intravenous'.",
    "An input route 'intramuscul' would be classified as 'intramuscular'.",
    "An input route 'rEcTal' would be classified as 'rectal'.",
    "An input route 'oral' would be classified as 'oral'.",
    "An input route 'po' would be classified as 'oral'.",
]

for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=1,
            agentic=False,
            n_shots=2,
      )
      
      results, decision, consistency = ehrm.get_generic_route(
            route='through the mouth',
            classes=['intravenous', 'rectal', 'oral','other'],
            possible_shots=possible_shots,
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: gpt-4o
Results: oral
Decision: 1.0


TypeError: unsupported format string passed to list.__format__

In [4]:
possible_shots = [
    "An input drugname 'vancocin' would be classified as 'vancomycin'.",
    "An input drugname 'vancomycin' would be classified as 'vancomycin'.",
    "An input drugname 'Toprol XL' would be classified as 'metroprolol'.",
    "An input drugname 'Adderall' would be classified as 'amphetamine/dextroamphetamine'.",
    "An input drugname 'Tylenol' would be classified as 'acetaminophen'.",
]

for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config(
            n_attempts=1,
            agentic=False,
            n_shots=2,
      )

      results, decision, consistency = ehrm.get_generic_name(
            drugname='ativan',
            possible_shots=possible_shots,
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")

Model: meta.llama3-70b-instruct-v1:0
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00


In [5]:
possible_shots = [
    "An input drugname 'vancocin' with a route 'intravenous' would be classified as 'antibiotic'.",
    "An input drugname 'vancomycin' with a route 'intravenous' would be classified as 'antibiotic'.",
    "An input drugname 'Toprol XL' with a route 'oral' would be classified as 'beta blocker'.",
    "An input drugname 'Tylenol' with a route 'oral' would be classified as 'analgesic'.",
    "An input drugname 'vasopressin' with a route 'intravenous' would be classified as 'vasopressor'.",
]

for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config(
            n_attempts=3,
            agentic=False,
            n_shots=2,
      )

      results, decision, consistency = ehrm.classify_drug(
            drugname='dobutamine',
            route='intravenous',
            classes=['antibiotic', 'anticoagulant', 'other'],
            possible_shots=possible_shots,
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: meta.llama3-70b-instruct-v1:0
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
